In [2]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx
import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re
# set some nicer defaults for matplotlib
from matplotlib import rcParams
from bs4 import BeautifulSoup

In [3]:
def get_poll_xml1(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com/draft/NHL_%i_entry.html" % int(poll_id)
    
    return requests.get(url).text

In [5]:
get_poll_xml1(2005)

u'<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/hr/build" itemscope itemtype="http://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/201609083" />\n    <link href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect" crossorigin>\n<link href="//d9kjk42l7bfqz.cloudfront.net/req/201607011" rel="preconnect" crossorigin>\n<link href="https://www.google-analytics.com/" rel="preconnect" crossorigin>\n<link href="https://www.googletagservices.com" rel="preconnect" crossorigin>\n\n\n\n   \n\n<script>var sr_is_production = true;\nvar log_performance=!1,sr_detect_operaMini=navigator.userAgent.indexOf("Opera Mini")>-1;if(sr_detect_operaMini){var el=document.querySelector("html");el.className=el.className.concat(" operamini")}var sr_detect_firefox=navigator.userAgent.indexOf("Firefox")>-1;if(sr_detec

In [6]:
soup = BeautifulSoup(get_poll_xml1(2005))

In [7]:
print soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/hr/build" data-version="klecko-" itemscope="" itemtype="http://schema.org/WebSite" lang="en">
<head>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201609083" rel="dns-prefetch"/>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="//d9kjk42l7bfqz.cloudfront.net/req/201607011" rel="preconnect"/>
<link crossorigin="" href="https://www.google-analytics.com/" rel="preconnect"/>
<link crossorigin="" href="https://www.googletagservices.com" rel="preconnect"/>
<script>var sr_is_production = true;
var log_performance=!1,sr_detect_operaMini=navigator.userAgent.indexOf("Opera Mini")>-1;if(sr_detect_operaMini){var el=document.querySelector("html");el.className=el.className.concat(" operamini")}var sr_detect_firefox=navigator.userAgent.indexOf("Firefox")>-1;if(sr_detect_firefox){var el=doc

In [8]:
play_list=[]
for link in soup.findAll('a'):
    cur_href=link.get('href')
    
    if len(cur_href)>9:
        if cur_href[1:8]=='players':
            play_list.append(cur_href)
            

In [222]:
play_list=[]

for year in range(2005,2006):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

In [227]:
play_list=play_list[:230]

In [228]:
len(play_list)

230

In [4]:
#http://www.hockey-reference.com/players/c/crosbsi01.html
def get_poll_xml12(poll_id):
    #url = "http://www.nhl.com/scores/htmlreports/20102011/GS0%i.HTM" % int(poll_id)
    url = "http://www.hockey-reference.com" + poll_id

    return requests.get(url).text

In [135]:
soup2 = BeautifulSoup(get_poll_xml12(play_list[0]))
soup2 = BeautifulSoup(get_poll_xml12(play_list[1]))

In [142]:
soup2

<!DOCTYPE html>
<html class="no-js" data-root="/home/hr/build" data-version="klecko-" itemscope="" itemtype="http://schema.org/WebSite" lang="en">
<head>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201609083" rel="dns-prefetch"/>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="//d9kjk42l7bfqz.cloudfront.net/req/201607011" rel="preconnect"/>
<link crossorigin="" href="https://www.google-analytics.com/" rel="preconnect"/>
<link crossorigin="" href="https://www.googletagservices.com" rel="preconnect"/>
<script>var sr_is_production = true;
var log_performance=!1,sr_detect_operaMini=navigator.userAgent.indexOf("Opera Mini")>-1;if(sr_detect_operaMini){var el=document.querySelector("html");el.className=el.className.concat(" operamini")}var sr_detect_firefox=navigator.userAgent.indexOf("Firefox")>-1;if(sr_detect_firefox){var el=doc

In [137]:
strsoup2=str(soup2)

In [6]:
def find_str(s, char):
    index = 0

    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index

            index += 1

    return -1

In [183]:
namePos=(find_str(strsoup2, "<h1 itemprop="))
namePosEnd=(find_str(strsoup2, "</h1>"))
heightPos=(find_str(strsoup2, "cm,"))
weightPos=(find_str(strsoup2, "lb<"))
shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
posPos=(find_str(strsoup2, "<strong>Position</strong>"))
draftPos=(find_str(strsoup2, "round ("))

In [186]:
name=strsoup2[(namePos+20):(namePosEnd)]
height=strsoup2[(heightPos-3):heightPos]
weight=strsoup2[(weightPos-3):weightPos]
shot=strsoup2[(shotPos+25):(shotPos+26)]
position=strsoup2[(posPos+27):(posPos+28)]
draft=strsoup2[(draftPos+7):(draftPos+10)]

In [188]:
print name
print play_list[1]
print int(height)
print int(weight)
print shot
print position
print draft

Bobby Ryan
/players/r/ryanbo01.html
188
209
R
L
2nd


In [189]:
playerdata=[]
playerdata.append(name)
playerdata.append(play_list[1])
playerdata.append(int(height))
playerdata.append(int(weight))
playerdata.append(shot)
playerdata.append(position)
playerdata.append(draft)
print playerdata

['Bobby Ryan', '/players/r/ryanbo01.html', 188, 209, 'R', 'L', '2nd']


In [193]:
dfplayer=pd.DataFrame([playerdata],columns=['name','url','height','weight','shot','pos','draft'])

In [194]:
dfplayer

,name,url,height,weight,shot,pos,draft
0,Bobby Ryan,/players/r/ryanbo01.html,188,209,R,L,2nd


In [199]:
play_list[10]

'/players/k/kopitan01.html'

In [5]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False


In [10]:
soup = BeautifulSoup(get_poll_xml1(2005))
play_list=[]

for year in range(2005,2006):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:230]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229


[['Wayne Gretzky', '/players/g/gretzwa01.html', '183', '185', 'L', 'C', 'YPE']]

In [235]:
dfplayer2005=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])

In [236]:
dfplayer2005.to_csv('dfplayer2005.csv')

In [243]:
soup = BeautifulSoup(get_poll_xml1(2007))
play_list=[]

for year in range(2007,(2007+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2007=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2007.to_csv('dfplayer2007.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [244]:
soup = BeautifulSoup(get_poll_xml1(2007))
play_list=[]

for year in range(2007,(2007+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2007=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2007.to_csv('dfplayer2007.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [245]:
soup = BeautifulSoup(get_poll_xml1(2008))
play_list=[]

for year in range(2008,(2008+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2008=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2008.to_csv('dfplayer2008.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [246]:
soup = BeautifulSoup(get_poll_xml1(2009))
play_list=[]

for year in range(2009,(2009+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2009=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2009.to_csv('dfplayer2009.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [247]:
soup = BeautifulSoup(get_poll_xml1(2010))
play_list=[]

for year in range(2010,(2010+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:211]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    #draft=strsoup2[(draftPos+7):(draftPos+10)]
    if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+8)])
    elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
        draft=int(strsoup2[(draftPos+7):(draftPos+9)])
    else:
        draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2010=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2010.to_csv('dfplayer2010.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


In [25]:
soup = BeautifulSoup(get_poll_xml1(2011))
play_list=[]

for year in range(2011,(2011+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2011=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2011.to_csv('dfplayer2011.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [250]:
soup = BeautifulSoup(get_poll_xml1(2012))
play_list=[]

for year in range(2012,(2012+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2012=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2012.to_csv('dfplayer2012.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [251]:
soup = BeautifulSoup(get_poll_xml1(2013))
play_list=[]

for year in range(2013,(2013+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2013=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2013.to_csv('dfplayer2013.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [252]:
soup = BeautifulSoup(get_poll_xml1(2014))
play_list=[]

for year in range(2014,(2014+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:211]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2014=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2014.to_csv('dfplayer2014.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


In [254]:
soup = BeautifulSoup(get_poll_xml1(2015))
play_list=[]

for year in range(2015,(2015+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:212]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2015=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2015.to_csv('dfplayer2015.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


In [255]:
soup = BeautifulSoup(get_poll_xml1(2004))
play_list=[]

for year in range(2004,(2004+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:292]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2004=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2004.to_csv('dfplayer2004.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [256]:
soup = BeautifulSoup(get_poll_xml1(2003))
play_list=[]

for year in range(2003,(2003+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:293]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2003=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2003.to_csv('dfplayer2003.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
soup = BeautifulSoup(get_poll_xml1(2002))
play_list=[]

for year in range(2002,(2002+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:292]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2002=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2002.to_csv('dfplayer2002.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [258]:
soup = BeautifulSoup(get_poll_xml1(2001))
play_list=[]

for year in range(2001,(2001+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:290]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2001=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2001.to_csv('dfplayer2001.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
# soup = BeautifulSoup(get_poll_xml1(2000))
# play_list=[]

# for year in range(2000,(2000+1)):
#     soup = BeautifulSoup(get_poll_xml1(year))
#     for link in soup.findAll('a'):
#         cur_href=link.get('href')

#         if len(cur_href)>9:
#             if cur_href[1:8]=='players':
#                 play_list.append(cur_href)

                
# play_list=play_list[:294]

# playerdata=[]
# print 'never'
# print play_list[0]
# # for i in range(len(play_list[10])):
# for i in range(1):
#     print i
#     #print play_list[i]
#     soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
#     strsoup2=str(soup2)
#     print soup2

In [21]:
soup = BeautifulSoup(get_poll_xml1(2000))
play_list=[]

for year in range(2000,(2000+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:294]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer2000=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer2000.to_csv('dfplayer2000.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139


In [22]:
dfplayer2000

,name,url,height,weight,shot,pos,draft
0,Rick DiPietro,/players/d/dipieri01.html,183,185,a,G,1st
1,Dany Heatley,/players/h/heatlda01.html,190,212,L,L,2nd
2,Marian Gaborik,/players/g/gaborma01.html,185,205,L,R,3rd
3,Rostislav Klesla,/players/k/kleslro01.html,190,223,L,D,4th
4,Raffi Torres,/players/t/torrera01.html,183,210,L,L,5th
5,Scott Hartnell,/players/h/hartnsc01.html,188,214,L,L,6th
6,Lars Jonsson,/players/j/jonssla01.html,185,205,L,D,7th
7,Nikita Alexeev,/players/a/alexeni01.html,196,227,L,R,8th
8,Brent Krahn,/players/k/krahnbr01.html,193,232,a,G,9th
9,Mikhail Yakubov,/players/y/yakubmi01.html,190,202,L,C,10t


In [23]:
soup = BeautifulSoup(get_poll_xml1(1999))
play_list=[]

for year in range(1999,(1999+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:273]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1999=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1999.to_csv('dfplayer1999.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133


In [263]:
soup = BeautifulSoup(get_poll_xml1(1998))
play_list=[]

for year in range(1998,(1998+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:259]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1998=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1998.to_csv('dfplayer1998.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [264]:
soup = BeautifulSoup(get_poll_xml1(1997))
play_list=[]

for year in range(1997,(1997+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:247]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1997=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1997.to_csv('dfplayer1997.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118


In [265]:
soup = BeautifulSoup(get_poll_xml1(1996))
play_list=[]

for year in range(1996,(1996+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:242]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1996=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1996.to_csv('dfplayer1996.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [266]:
soup = BeautifulSoup(get_poll_xml1(1995))
play_list=[]

for year in range(1995,(1995+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:235]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1995=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1995.to_csv('dfplayer1995.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130


In [267]:
soup = BeautifulSoup(get_poll_xml1(1994))
play_list=[]

for year in range(1994,(1994+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:287]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1994=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1994.to_csv('dfplayer1994.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132


In [268]:
soup = BeautifulSoup(get_poll_xml1(1993))
play_list=[]

for year in range(1993,(1993+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:287]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1993=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1993.to_csv('dfplayer1993.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148


In [269]:
soup = BeautifulSoup(get_poll_xml1(1992))
play_list=[]

for year in range(1992,(1992+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:265]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1992=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1992.to_csv('dfplayer1992.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142


In [270]:
soup = BeautifulSoup(get_poll_xml1(1991))
play_list=[]

for year in range(1991,(1991+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:265]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1991=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1991.to_csv('dfplayer1991.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130


In [271]:
soup = BeautifulSoup(get_poll_xml1(1990))
play_list=[]

for year in range(1990,(1990+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:253]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1990=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1990.to_csv('dfplayer1990.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


In [272]:
soup = BeautifulSoup(get_poll_xml1(1989))
play_list=[]

for year in range(1989,(1989+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:253]

playerdata=[]

# for i in range(len(play_list[10])):
for i in range(len(play_list)):
    print i
    #print play_list[i]
    soup2 = BeautifulSoup(get_poll_xml12(play_list[i]))
    strsoup2=str(soup2)
    namePos=(find_str(strsoup2, "<h1 itemprop="))
    namePosEnd=(find_str(strsoup2, "</h1>"))
    heightPos=(find_str(strsoup2, "cm,"))
    weightPos=(find_str(strsoup2, "lb<"))
    shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
    posPos=(find_str(strsoup2, "<strong>Position</strong>"))
    draftPos=(find_str(strsoup2, "round ("))
    
    name=strsoup2[(namePos+20):(namePosEnd)]
    height=strsoup2[(heightPos-3):heightPos]
    weight=strsoup2[(weightPos-3):weightPos]
    shot=strsoup2[(shotPos+25):(shotPos+26)]
    position=strsoup2[(posPos+27):(posPos+28)]
    #print strsoup2[(draftPos+7):(draftPos+10)]
    draft=strsoup2[(draftPos+7):(draftPos+10)]
#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
        
    #RepresentsInt("+123")
    playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
dfplayer1989=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
dfplayer1989.to_csv('dfplayer1989.csv')    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130


In [8]:
http://www.hockey-reference.com/players/b/bennebe01.html

'/players/d/dipieri01.html'

In [34]:
soup = BeautifulSoup(get_poll_xml1(2000))
play_list=[]

for year in range(2000,(2000+1)):
    soup = BeautifulSoup(get_poll_xml1(year))
    for link in soup.findAll('a'):
        cur_href=link.get('href')

        if len(cur_href)>9:
            if cur_href[1:8]=='players':
                play_list.append(cur_href)

                
play_list=play_list[:294]

playerdata=[]

# for i in range(len(play_list[10])):
# for i in range(len(play_list[0])):
#     print i
#     #print play_list[i]
soup2 = BeautifulSoup(get_poll_xml12(play_list[0]))
strsoup2=str(soup2)
print soup2
# namePos=(find_str(strsoup2, "<h1 itemprop="))
# namePosEnd=(find_str(strsoup2, "</h1>"))
# heightPos=(find_str(strsoup2, "cm,"))
# weightPos=(find_str(strsoup2, "lb<"))
# shotPos=(find_str(strsoup2, "<strong>Shoots</strong>:"))
# posPos=(find_str(strsoup2, "<strong>Position</strong>"))
# draftPos=(find_str(strsoup2, "round ("))

# name=strsoup2[(namePos+20):(namePosEnd)]
# height=strsoup2[(heightPos-3):heightPos]
# weight=strsoup2[(weightPos-3):weightPos]
# shot=strsoup2[(shotPos+25):(shotPos+26)]
# position=strsoup2[(posPos+27):(posPos+28)]
# #print strsoup2[(draftPos+7):(draftPos+10)]
# draft=strsoup2[(draftPos+7):(draftPos+10)]






#     if RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][1])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+8)])
#     elif RepresentsInt(strsoup2[(draftPos+7):(draftPos+10)][2])==False:
#         draft=int(strsoup2[(draftPos+7):(draftPos+9)])
#     else:
#         draft=int(strsoup2[(draftPos+7):(draftPos+10)])
        
        
resultEven1= soup.find_all("tr", attrs={"id":"stats_basic_plus_nhl.2001"} )

    #RepresentsInt("+123")
#     playerdata.append([name,play_list[i],height,weight,shot,position,draft])
    
# dfplayer2000=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
# dfplayer2000.to_csv('dfplayer2000.csv')    

<!DOCTYPE html>
<html class="no-js" data-root="/home/hr/build" data-version="klecko-" itemscope="" itemtype="http://schema.org/WebSite" lang="en">
<head>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201609083" rel="dns-prefetch"/>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="//d9kjk42l7bfqz.cloudfront.net/req/201607011" rel="preconnect"/>
<link crossorigin="" href="https://www.google-analytics.com/" rel="preconnect"/>
<link crossorigin="" href="https://www.googletagservices.com" rel="preconnect"/>
<script>var sr_is_production = true;
var log_performance=!1,sr_detect_operaMini=navigator.userAgent.indexOf("Opera Mini")>-1;if(sr_detect_operaMini){var el=document.querySelector("html");el.className=el.className.concat(" operamini")}var sr_detect_firefox=navigator.userAgent.indexOf("Firefox")>-1;if(sr_detect_firefox){var el=doc

In [53]:
# table = soup.find('table')
table = soup.find('table_outer_container')

In [54]:
table

In [51]:
table_body = table.find_all('tbody')

In [52]:
table_body 

[<tbody><tr><th class="right " data-stat="pick_overall" scope="row">1</th><td class="left " csk="NYI.001" data-stat="team_name"><a href="/teams/NYI/draft.html">New York Islanders</a></td><td class="left " data-append-csv="dipieri01" data-stat="player"><a href="/players/d/dipieri01.html">Rick DiPietro</a></td><td class="center " data-stat="birth_country">US</td><td class="center " csk="G.001" data-stat="pos">G</td><td class="center " data-stat="draft_age">18</td><td class="right " data-stat="year_last">2013</td><td class="left " data-stat="amateur_team">Boston University (H-East)</td><td class="right " data-stat="games_played">318</td><td class="right " data-stat="goals">0</td><td class="right " data-stat="assists">19</td><td class="right " data-stat="points">19</td><td class="right " data-stat="plus_minus">0</td><td class="right " data-stat="pen_min">129</td><td class="right " data-stat="games_goalie">318</td><td class="right " data-stat="wins_goalie">130</td><td class="right " data-st

In [60]:
soup.find_all(attrs={"class":"table_outer_container"})

[<div class="table_outer_container">
 <div class="overthrow table_container" id="div_stats">
 <table class="sortable stats_table" data-cols-to-freeze="1" id="stats"><caption>293 Selections Table</caption>
 <colgroup><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col><col></col></colgroup>
 <thead>
 <tr class="over_header"><th></th>
 <th aria-label="" class=" over_header " colspan="2" data-stat="header_draft">Round 1</th>
 <th aria-label="" class=" over_header " colspan="4" data-stat=""></th><th></th>
 <th aria-label="" class=" over_header center" colspan="6" data-stat="header_skaters">NHL Scoring Stats</th>
 <th aria-label="" class=" over_header center" colspan="6" data-stat="header_goalies">NHL Goalie Stats</th>
 </tr>
 <tr>
 <th aria-label="Overall" class=" tooltip sort_default_asc center" data-stat="pick_overall" scope="col">Ov

In [62]:
# soup.find_all(id=="stats_basic_plus_nhl.2001")

In [30]:
namePos=(find_str(strsoup2, "save_pct"))

In [31]:
namePos

40703

In [56]:
table_body = soup.find('tbody')

In [57]:
table_body

<tbody><tr><th class="right " data-stat="pick_overall" scope="row">1</th><td class="left " csk="NYI.001" data-stat="team_name"><a href="/teams/NYI/draft.html">New York Islanders</a></td><td class="left " data-append-csv="dipieri01" data-stat="player"><a href="/players/d/dipieri01.html">Rick DiPietro</a></td><td class="center " data-stat="birth_country">US</td><td class="center " csk="G.001" data-stat="pos">G</td><td class="center " data-stat="draft_age">18</td><td class="right " data-stat="year_last">2013</td><td class="left " data-stat="amateur_team">Boston University (H-East)</td><td class="right " data-stat="games_played">318</td><td class="right " data-stat="goals">0</td><td class="right " data-stat="assists">19</td><td class="right " data-stat="points">19</td><td class="right " data-stat="plus_minus">0</td><td class="right " data-stat="pen_min">129</td><td class="right " data-stat="games_goalie">318</td><td class="right " data-stat="wins_goalie">130</td><td class="right " data-sta

In [55]:
soup.find_all(id="stats_basic_plus_nhl.2001")

[]

In [25]:
a.find_all(id="stats_basic_plus_nhl.2001")

AttributeError: 'ResultSet' object has no attribute 'find_all'

In [13]:
soup.find_all("th", attrs={"class":"left "} )

[]

In [ ]:
http://www.hockey-reference.com/players/k/kanepa01.html

In [26]:
url = "http://www.hockey-reference.com/players/k/kanepa01.html" 
page=requests.get(url).text

In [39]:
indexLoc= [i for i,x in enumerate(url) if x == '/']
indexDot= [i for i,x in enumerate(url) if x == '.']

In [40]:
indexDot

[10, 27, 50]

In [44]:
url[(indexLoc[-1]+1):(indexDot[-1])]

'kanepa01'

In [27]:
page

u'<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/hr/build" itemscope itemtype="http://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/201609083" />\n    <link href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect" crossorigin>\n<link href="//d9kjk42l7bfqz.cloudfront.net/req/201607011" rel="preconnect" crossorigin>\n<link href="https://www.google-analytics.com/" rel="preconnect" crossorigin>\n<link href="https://www.googletagservices.com" rel="preconnect" crossorigin>\n\n\n\n   \n\n<script>var sr_is_production = true;\nvar log_performance=!1,sr_detect_operaMini=navigator.userAgent.indexOf("Opera Mini")>-1;if(sr_detect_operaMini){var el=document.querySelector("html");el.className=el.className.concat(" operamini")}var sr_detect_firefox=navigator.userAgent.indexOf("Firefox")>-1;if(sr_detec

In [8]:
pagesoap=BeautifulSoup(page)

In [9]:
strsoap=str(pagesoap)

In [10]:
posseStr=(find_str(strsoap, "Defensive Zone Faceoffs / (Offensive Zone Faceoffs + Defensive Zone Faceoffs),<br />that took place while on the ice"))

In [11]:
posStr=(find_str(strsoap, "Awards</th>\n</tr>\n</thead>\n<tbody>"))

In [12]:
posEnd=(find_str(strsoap, "</tr>\n</tfoot>\n</table>\n</div>\n</div>\n</div><!-- div.table_wrapper -->\n<!-- sr_mobile_1 -->"))

In [13]:
# <h2>NHL Miscellaneous</h2>
posseEnd=(find_str(strsoap, " <h2>NHL Miscellaneous</h2>"))


In [14]:
strlimitPos=strsoap[(posseStr+116):posseEnd]
strlimit=strsoap[(posStr+34):posEnd]

In [15]:
for m in re.finditer('season', strlimitPos):     
    if strlimitPos[(m.end()+3):(m.end()+5)]=='Ca':
        break
    
    else:
        print strlimitPos[(m.end()+3):(m.end()+10)]
        

2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16


In [18]:
age=['age']
season=['season']
corsi_for=['corsi_for']
corsi_against=['corsi_against']
corsi_pct=['corsi_pct']
corsi_rel_pct=['corsi_rel_pct']
fenwick_for=['fenwick_for']
fenwick_against=['fenwick_against']
fenwick_pct=['fenwick_pct']
fenwick_rel_pct=['fenwick_rel_pct']
on_ice_shot_pct=['on_ice_shot_pct']
on_ice_sv_pct=['on_ice_sv_pct']
pdo=['pdo']
zs_offense_pct=['zs_offense_pct']
zs_defense_pct=['zs_defense_pct']

for m in re.finditer('season', strlimitPos):     
    if strlimitPos[(m.end()+3):(m.end()+5)]=='Ca':
        break
    else:
        season.append(strlimitPos[(m.end()+3):(m.end()+10)])

for m in re.finditer('age', strlimitPos):
    if strlimitPos[(m.end()+3):(m.end()+5)]=='</':
        break
    else:
        age.append(strlimitPos[(m.end()+3):(m.end()+5)])
                   
for m in re.finditer('corsi_for', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        corsi_for.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        corsi_for.append(strlimitPos[(m.end()+3):(m.end()+7)])

for m in re.finditer('corsi_against', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        corsi_against.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        corsi_against.append(strlimitPos[(m.end()+3):(m.end()+7)])
        
for m in re.finditer('corsi_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        corsi_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        corsi_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])
        
for m in re.finditer('corsi_rel_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        corsi_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        corsi_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])
        
for m in re.finditer('fenwick_for', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        fenwick_for.append(strlimitPos[(m.end()+3):(m.end()+7)])
        
for m in re.finditer('fenwick_against', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        fenwick_against.append(strlimitPos[(m.end()+3):(m.end()+7)])        
        
        
for m in re.finditer('fenwick_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        fenwick_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        fenwick_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        

for m in re.finditer('fenwick_rel_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        fenwick_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        fenwick_rel_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        

for m in re.finditer('on_ice_shot_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        on_ice_shot_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        on_ice_shot_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])
        
for m in re.finditer('on_ice_sv_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        on_ice_sv_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        on_ice_sv_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        
        
for m in re.finditer('pdo', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        pdo.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        pdo.append(strlimitPos[(m.end()+3):(m.end()+7)])        
        
for m in re.finditer('zs_offense_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        zs_offense_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        zs_offense_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])        
        
for m in re.finditer('zs_defense_pct', strlimitPos):
    if strlimitPos[(m.end()+6)]=='<':
        zs_defense_pct.append(strlimitPos[(m.end()+3):(m.end()+6)])
    else:
        zs_defense_pct.append(strlimitPos[(m.end()+3):(m.end()+7)])         
        
        
matrix=[]        
matrix.append(corsi_for[:-1])
matrix.append(corsi_against[:-1])
matrix.append(corsi_pct[:-1])
matrix.append(corsi_rel_pct[:-1])
matrix.append(fenwick_for[:-1])
matrix.append(fenwick_against[:-1])
matrix.append(fenwick_pct[:-1])
matrix.append(fenwick_rel_pct[:-1])

matrix.append(on_ice_shot_pct[:-1])
matrix.append(on_ice_sv_pct[:-1])
matrix.append(pdo[:-1])

matrix.append(zs_offense_pct[:-1])
matrix.append(zs_defense_pct[:-1])
matrix

[['corsi_for',
  '963',
  '1125',
  '1353',
  '1213',
  '1390',
  '706',
  '1155',
  '1044',
  '1397'],
 ['corsi_against',
  '866',
  '874',
  '947',
  '927',
  '1140',
  '684',
  '924',
  '863',
  '1235'],
 ['corsi_pct',
  '52.7',
  '56.3',
  '58.8',
  '56.7',
  '54.9',
  '50.8',
  '55.6',
  '54.7',
  '53.1'],
 ['corsi_rel_pct',
  '1.5',
  '0.1',
  '0.8',
  '2.5',
  '2.1',
  '-6.7',
  '-2.4',
  '0.3',
  '3.3'],
 ['fenwick_for',
  '747',
  '863',
  '1051',
  '913',
  '1033',
  '534',
  '863',
  '771',
  '990'],
 ['fenwick_against',
  '688',
  '698',
  '703',
  '714',
  '885',
  '494',
  '697',
  '663',
  '930'],
 ['fenwick_pct',
  '52.1',
  '55.3',
  '59.9',
  '56.1',
  '53.9',
  '51.9',
  '55.3',
  '53.8',
  '51.6'],
 ['fenwick_rel_pct',
  '0.7',
  '-0.8',
  '0.1',
  '2.0',
  '0.6',
  '-7.4',
  '-2.6',
  '0.1',
  '1.9'],
 ['on_ice_shot_pct',
  '9.9',
  '7.5',
  '9.1',
  '9.5',
  '9.6',
  '11.9',
  '8.9',
  '9.4',
  '11.2'],
 ['on_ice_sv_pct',
  '89.2',
  '91.4',
  '90.2',
  '89.8',
  

In [48]:
games_played=['games_played']
goals=['goals']
assists=['assists']
points=['points']
plus_minus=['plus_minus']
pen_min=['pen_min']
goals_ev=['goals_ev']
goals_pp=['goals_pp']
goals_sh=['goals_sh']
goals_gw=['goals_gw']
assists_ev=['assists_ev']
assists_pp=['assists_pp']
assists_sh=['assists_sh']
shots=['shots']
shot_pct=['shot_pct']
shots_attempted=['shots_attempted']
time_on_ice=['time_on_ice']
time_on_ice_avg=['time_on_ice_avg']
faceoff_wins=[]
faceoff_losses=[]
faceoff_percentage=[]
hits=['hits']
blocks=['blocks']
takeaways=['takeaways']
giveaways=['giveaways']
award_summary=['award_summary']


for m in re.finditer('games_played', strlimit):
    games_played.append(strlimit[(m.end()+2):(m.end()+4)])
    
for m in re.finditer('goals_ev', strlimit):
    if strlimit[(m.end()+3)]=='<':
        goals_ev.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        goals_ev.append(strlimit[(m.end()+2):(m.end()+4)])
    
for m in re.finditer('goals_pp', strlimit):
    if strlimit[(m.end()+3)]=='<':
        goals_pp.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        goals_pp.append(strlimit[(m.end()+2):(m.end()+4)])


for m in re.finditer('goals_sh', strlimit):
    if strlimit[(m.end()+3)]=='<':
        goals_sh.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        goals_sh.append(strlimit[(m.end()+2):(m.end()+4)])

for m in re.finditer('goals_gw', strlimit):
    if strlimit[(m.end()+3)]=='<':
        goals_gw.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        goals_gw.append(strlimit[(m.end()+2):(m.end()+4)])

        
for m in re.finditer('assists_ev', strlimit):
    if strlimit[(m.end()+3)]=='<':
        assists_ev.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        assists_ev.append(strlimit[(m.end()+2):(m.end()+4)])
        
for m in re.finditer('assists_pp', strlimit):
    if strlimit[(m.end()+3)]=='<':
        assists_pp.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        assists_pp.append(strlimit[(m.end()+2):(m.end()+4)])
        
for m in re.finditer('assists_sh', strlimit):
    if strlimit[(m.end()+3)]=='<':
        assists_sh.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        assists_sh.append(strlimit[(m.end()+2):(m.end()+4)])

for m in re.finditer('shot_pct', strlimit):
    if strlimit[(m.end()+5)]=='<':
        shot_pct.append(strlimit[(m.end()+2):(m.end()+5)])
    else:
        shot_pct.append(strlimit[(m.end()+2):(m.end()+6)])
        
for m in re.finditer('shots_attempted', strlimit):
    shots_attempted.append(strlimit[(m.end()+2):(m.end()+5)])

for m in re.finditer('time_on_ice', strlimit):
    time_on_ice.append(strlimit[(m.end()+2):(m.end()+4)])    

for m in re.finditer('time_on_ice_avg', strlimit):
    time_on_ice_avg.append(strlimit[(m.end()+2):(m.end()+7)])  

    
for m in re.finditer('hits', strlimit):
    if strlimit[(m.end()+4)]=='<':
        hits.append(strlimit[(m.end()+2):(m.end()+4)])
    else:
        hits.append(strlimit[(m.end()+2):(m.end()+5)])
    
for m in re.finditer('blocks', strlimit):
    if strlimit[(m.end()+3)]=='<':
        blocks.append(strlimit[(m.end()+2):(m.end()+3)])
    else:
        blocks.append(strlimit[(m.end()+2):(m.end()+4)])
    
for m in re.finditer('takeaways', strlimit):
    takeaways.append(strlimit[(m.end()+2):(m.end()+4)])
    
for m in re.finditer('giveaways', strlimit):
    giveaways.append(strlimit[(m.end()+2):(m.end()+4)])
    

for m in re.finditer('plus_minus', strlimit):
    if strlimit[(m.end()+3)]=='<':
        plus_minus.append(strlimit[(m.end()+2):(m.end()+3)])
    elif strlimit[(m.end()+4)]=='<':
        plus_minus.append(strlimit[(m.end()+2):(m.end()+4)])
    else:
        plus_minus.append(strlimit[(m.end()+2):(m.end()+5)])
    
for m in re.finditer('pen_min', strlimit):
    if strlimit[(m.end()+3)]=='<':
        pen_min.append(strlimit[(m.end()+2):(m.end()+3)])
    elif strlimit[(m.end()+4)]=='<':
        pen_min.append(strlimit[(m.end()+2):(m.end()+4)])
    elif strlimit[(m.end()+5)]=='<':
        pen_min.append(strlimit[(m.end()+2):(m.end()+5)])
    else:
        pen_min.append(strlimit[(m.end()+2):(m.end()+6)])

matrix.append(games_played[:-1])
matrix.append(pen_min[:-1])
matrix.append(plus_minus[:-1])
matrix.append(goals_ev[:-1])
matrix.append(goals_pp[:-1])
matrix.append(goals_sh[:-1])
matrix.append(goals_gw[:-1])

matrix.append(assists_ev[:-1])
matrix.append(assists_pp[:-1])
matrix.append(assists_sh[:-1])

matrix.append(shot_pct[:-1])
matrix.append(shots_attempted[:-1])

matrix.append(shot_pct[:-1])
matrix.append(shots_attempted[:-1])
matrix.append(time_on_ice_avg[:-1])
matrix.append(hits[:-1])

matrix.append(blocks[:-1])
matrix.append(takeaways[:-1])
matrix.append(giveaways[:-1])

newmatrix=[list(i) for i in zip(*matrix)]

newdf=pd.DataFrame(newmatrix)
namecsv=url[(indexLoc[-1]+1):(indexDot[-1])]+".csv"
newdf.to_csv(namecsv)  

[['corsi_for',
  '963',
  '1125',
  '1353',
  '1213',
  '1390',
  '706',
  '1155',
  '1044',
  '1397'],
 ['corsi_against',
  '866',
  '874',
  '947',
  '927',
  '1140',
  '684',
  '924',
  '863',
  '1235'],
 ['corsi_pct',
  '52.7',
  '56.3',
  '58.8',
  '56.7',
  '54.9',
  '50.8',
  '55.6',
  '54.7',
  '53.1'],
 ['corsi_rel_pct',
  '1.5',
  '0.1',
  '0.8',
  '2.5',
  '2.1',
  '-6.7',
  '-2.4',
  '0.3',
  '3.3'],
 ['fenwick_for',
  '747',
  '863',
  '1051',
  '913',
  '1033',
  '534',
  '863',
  '771',
  '990'],
 ['fenwick_against',
  '688',
  '698',
  '703',
  '714',
  '885',
  '494',
  '697',
  '663',
  '930'],
 ['fenwick_pct',
  '52.1',
  '55.3',
  '59.9',
  '56.1',
  '53.9',
  '51.9',
  '55.3',
  '53.8',
  '51.6'],
 ['fenwick_rel_pct',
  '0.7',
  '-0.8',
  '0.1',
  '2.0',
  '0.6',
  '-7.4',
  '-2.6',
  '0.1',
  '1.9'],
 ['on_ice_shot_pct',
  '9.9',
  '7.5',
  '9.1',
  '9.5',
  '9.6',
  '11.9',
  '8.9',
  '9.4',
  '11.2'],
 ['on_ice_sv_pct',
  '89.2',
  '91.4',
  '90.2',
  '89.8',
  

'kanepa01'

In [45]:
#dfplayer2007=pd.DataFrame(playerdata,columns=['name','url','height','weight','shot','pos','draft'])
newdf=pd.DataFrame(newmatrix)
namecsv=url[(indexLoc[-1]+1):(indexDot[-1])]+".csv"
newdf.to_csv(namecsv)  